# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from google_key import g_key

gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = os.path.join('..', 'WeatherPy', 'weather.csv')
weather = pd.read_csv(weather_df)
weather=weather.dropna()
cityW=pd.DataFrame(weather).rename(columns={"Unnamed: 0":"City"})
cityW

,City,Clouds,Humidity,Lat,Lon,Temp,Wind
0,mehamn,75.0,72.0,71.04,27.85,17.60,77.17
1,ponta do sol,20.0,68.0,32.67,-17.10,60.13,8.05
2,aksu,0.0,65.0,41.12,80.26,22.69,1.59
3,kirakira,100.0,81.0,-10.45,161.92,80.62,4.25
4,surin,100.0,85.0,14.88,103.49,75.99,5.19
...,...,...,...,...,...,...,...
576,coahuayana,20.0,50.0,18.73,-103.68,77.00,8.81
577,adrar,0.0,20.0,20.50,-10.07,75.51,11.32
578,skibbereen,75.0,65.0,51.55,-9.27,35.60,14.99
579,enshi,99.0,97.0,30.30,109.48,30.45,0.74


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather[["Lat", "Lon"]]
hum = weather["Humidity"].astype(float)

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=hum, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_temp = weather[weather['Temp']>70]
best_temp = ideal_temp[ideal_temp['Temp']<80]
wind_speed = best_temp[best_temp['Wind']<10]
no_clouds = wind_speed[wind_speed['Clouds']==0]
no_clouds.head(50)

cityFilter=cityW.loc[(cityW.Temp>70) & (cityW.Temp<80) & (cityW.Clouds==0) & (cityW.Wind<10)]
cityFilter

,City,Clouds,Humidity,Lat,Lon,Temp,Wind
20,dubbo,0.0,78.0,-32.25,148.62,75.20,8.05
58,saint-philippe,0.0,73.0,-21.36,55.77,70.72,5.82
90,desdunes,0.0,56.0,19.30,-72.65,76.60,6.42
142,rawson,0.0,43.0,-43.30,-65.10,73.94,3.74
202,beloha,0.0,81.0,-25.17,45.05,70.59,1.41
264,salina cruz,0.0,76.0,16.17,-95.20,78.96,7.11
265,alice springs,0.0,20.0,-23.70,133.88,77.00,5.82
340,diapaga,0.0,11.0,12.07,1.79,73.36,7.58
438,gigmoto,0.0,82.0,13.78,124.39,78.93,9.95
482,saint-pierre,0.0,73.0,-21.34,55.48,70.79,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:

hotel_df=cityFilter[["City","Lat","Lon"]].copy()
hotel_df["Hotel"]=""
hotel_df.head()

,City,Lat,Lon,Hotel
20,dubbo,-32.25,148.62,
58,saint-philippe,-21.36,55.77,
90,desdunes,19.30,-72.65,
142,rawson,-43.30,-65.10,
202,beloha,-25.17,45.05,


In [20]:
params = {
    "radius":5000,
    "types":"lodging",
    "key":g_key
}

for index,row in hotel_df.iterrows():
    lat=row.Lat
    lng=row.Lon
    # Build the endpoint URL
    target_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params["location"]=f"{lat},{lng}"
    res=requests.get(target_url,params)
    res=res.json()
    try:
        hotel_df.loc[index,"Hotel"]=res["results"][0]["name"]
    except(KeyError,IndexError):
        print("missing field/results... Skipping.")


missing field/results... Skipping.
missing field/results... Skipping.


In [21]:
hotel_df

,City,Lat,Lon,Hotel
20,dubbo,-32.25,148.62,Comfort Inn Blue Lagoon
58,saint-philippe,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
90,desdunes,19.30,-72.65,Ozanana Hotel
142,rawson,-43.30,-65.10,Hotel Punta León
202,beloha,-25.17,45.05,
264,salina cruz,16.17,-95.20,Hotel Amerik
265,alice springs,-23.70,133.88,Desert Palms Alice Springs
340,diapaga,12.07,1.79,
438,gigmoto,13.78,124.39,Cj's Travellers Inn
482,saint-pierre,-21.34,55.48,Lindsey Hôtel


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]
locations

,Lat,Lon
20,-32.25,148.62
58,-21.36,55.77
90,19.30,-72.65
142,-43.30,-65.10
202,-25.17,45.05
264,16.17,-95.20
265,-23.70,133.88
340,12.07,1.79
438,13.78,124.39
482,-21.34,55.48


In [27]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))